In [13]:
import yfinance as yf
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ticker = "AAPL"
df = yf.download(ticker, period="8d", interval="1m")
df = df[['Close', 'Volume']].dropna()
data = df.to_numpy()

sequence_length = 60

X, y_data = [], []
for i in range(len(data) - sequence_length):
    X.append(data[i:i + sequence_length])
    y_data.append(data[i + 1:i + sequence_length + 1])
X = np.array(X)
y_data = np.array(y_data)

print("Sliding window input shape:", X.shape)
print("Sliding window output shape:", y_data.shape)

test_size = 360
train_X = X[:-test_size]
train_y = y_data[:-test_size]
test_X = X[-test_size:]
test_y = y_data[-test_size:]

# Convert to tensors and immediately move them to GPU
train_X_tensor = torch.tensor(train_X, dtype=torch.float32, device=device)
train_y_tensor = torch.tensor(train_y, dtype=torch.float32, device=device)
test_X_tensor = torch.tensor(test_X, dtype=torch.float32, device=device)
test_y_tensor = torch.tensor(test_y, dtype=torch.float32, device=device)

# Create datasets using the GPU-resident tensors
train_dataset = TensorDataset(train_X_tensor, train_y_tensor)
test_dataset = TensorDataset(test_X_tensor, test_y_tensor)

batch_size = 64
# Use num_workers=0 to avoid issues with GPU memory sharing between subprocesses.
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)


[*********************100%***********************]  1 of 1 completed

Sliding window input shape: (3060, 60, 2)
Sliding window output shape: (3060, 60, 2)


In [14]:
import torch
from torch import nn


class TransformerBlock(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int, mlp_hidden_dim: int, dropout: float):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_hidden_dim),
            nn.ReLU(),
            nn.Linear(mlp_hidden_dim, embed_dim)
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = x + self.dropout(attn_output)
        x = self.norm1(x)

        mlp_output = self.mlp(x)
        x = x + self.dropout(mlp_output)
        x = self.norm2(x)

        return x


class Model(nn.Module):
    def __init__(self, num_attention_blocks: int, num_heads: int, embed_dim: int, mlp_hidden_dim: int, dropout: float):
        super().__init__()

        self.input_projection = nn.Linear(2, embed_dim)

        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_hidden_dim, dropout)
            for _ in range(num_attention_blocks)
        ])

        self.output_projection = nn.Linear(embed_dim, 2)

    def forward(self, x):
        x = self.input_projection(x)

        for block in self.blocks:
            x = block(x)

        out = self.output_projection(x)
        return out

In [15]:
model = Model(
    num_attention_blocks=4,
    num_heads=4,
    embed_dim=128,
    mlp_hidden_dim=512,
    dropout=0.1
).to("cuda")

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{EPOCHS} - Loss: {avg_loss:.4f}")

Epoch 1/10 - Loss: 14951284902.6977
Epoch 2/10 - Loss: 15125461813.5814
Epoch 3/10 - Loss: 15008143705.3023
Epoch 4/10 - Loss: 14927678404.4651
Epoch 5/10 - Loss: 14961304052.0930
Epoch 6/10 - Loss: 14979070666.4186
Epoch 7/10 - Loss: 15123576379.5349
Epoch 8/10 - Loss: 14922584409.3023
Epoch 9/10 - Loss: 14813283339.9070
Epoch 10/10 - Loss: 14933272314.0465
